In [ ]:

!pip install gradio langchain faiss-cpu PyPDF2 sentence-transformers python-dotenv tiktoken
!pip install -U langchain-community

import gradio as gr
import os
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.docstore.document import Document
from langchain.llms.base import LLM
from typing import List, Optional
import requests
import warnings
warnings.filterwarnings("ignore")



TOGETHER_API_KEY = "bf14ef12c8d2b063ddadc885fce6562540304efd9d9c4372df942782a626816b"
TOGETHER_API_URL = "https://api.together.xyz/v1/chat/completions"


class TogetherLLM(LLM):
    model: str = "mistralai/Mixtral-8x7B-Instruct-v0.1"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Authorization": f"Bearer {TOGETHER_API_KEY}",
            "Content-Type": "application/json",
        }

        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.7
        }

        response = requests.post(TOGETHER_API_URL, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()['choices'][0]['message']['content'].strip()
        else:
            raise Exception(f"Together.ai error: {response.text}")

    @property
    def _llm_type(self) -> str:
        return "together_custom"


class PDFQABot:
    def __init__(self):
        self.vector_store = None
        self.documents = []
        self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        self.llm = TogetherLLM()
        self.qa_chain = load_qa_chain(self.llm, chain_type="stuff")

    def extract_text_from_pdf(self, pdf_file) -> str:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text

    def process_pdf(self, pdf_file) -> str:
        if not TOGETHER_API_KEY or TOGETHER_API_KEY == "your-together-api-key":
            return "❌ Please set your Together.ai API key first."

        text = self.extract_text_from_pdf(pdf_file)
        if not text.strip():
            return "❌ No text found in the PDF."

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = text_splitter.split_text(text)
        self.documents = [Document(page_content=chunk) for chunk in chunks]

        self.vector_store = FAISS.from_documents(self.documents, self.embeddings)
        return f"✅ PDF processed successfully! Created {len(chunks)} chunks."

    def ask_question(self, question: str) -> str:
        if not self.vector_store:
            return "❌ Please upload and process a PDF first."
        if not question.strip():
            return "❌ Please enter a question."

        docs = self.vector_store.similarity_search(question, k=3)
        answer = self.qa_chain.run(input_documents=docs, question=question)
        return f"**Answer:** {answer}"

    def get_document_info(self) -> str:
        if not self.documents:
            return "No document loaded."
        total_chars = sum(len(doc.page_content) for doc in self.documents)
        return f"Document has {len(self.documents)} chunks and {total_chars} total characters."

bot = PDFQABot()

def create_interface():
    with gr.Blocks(title="PDF Q&A Bot - Together.ai") as demo:
        gr.Markdown("""
        # 🤖 PDF Q&A Bot (Together.ai)
        Upload a PDF and ask AI questions based on its content.
        """)

        with gr.Row():
            with gr.Column():
                pdf_upload = gr.File(label="Upload PDF", file_types=[".pdf"], type="filepath")
                process_btn = gr.Button("Process PDF")
                process_status = gr.Textbox(label="Status", interactive=False)
                doc_info = gr.Textbox(label="Document Info", interactive=False)

            with gr.Column():
                question_input = gr.Textbox(label="Ask a Question")
                ask_btn = gr.Button("Ask")
                answer_output = gr.Textbox(label="Answer", interactive=False, lines=10)

        process_btn.click(fn=lambda f: (bot.process_pdf(f), bot.get_document_info()), inputs=pdf_upload, outputs=[process_status, doc_info])
        ask_btn.click(fn=bot.ask_question, inputs=question_input, outputs=answer_output)
        question_input.submit(fn=bot.ask_question, inputs=question_input, outputs=answer_output)

    return demo

if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True, server_name="0.0.0.0", server_port=7860)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6226ca9af31ffd655e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
